In [7]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

warnings.filterwarnings('ignore')

In [8]:
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

In [9]:
# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    # Leave only relevant store
    df = df[df['store_id']==store]

    # With memory limits we have to read 
    # lags and mean encoding features
    # separately and drop items that we don't need.
    # As our Features Grids are aligned 
    # we can use index to keep only necessary rows
    # Alignment is good for us as concat uses less memory than merge.
    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2 # to not reach memory limit 
    
    df = pd.concat([df, df3], axis=1)
    del df3 # to not reach memory limit 
    
    if store_id in ['CA_1', 'CA_2', 'CA_3','CA_4','TX_1','TX_2','TX_3']:
        remove_features = ['id','state_id','store_id','date','wm_yr_wk','d',TARGET,'cluster','snow_m',
                          'rolling_quantile_97_28', 'rolling_quantile_87.5_28', 'rolling_quantile_50_28', 'rolling_quantile_22.5_28', 'rolling_quantile_3_28', 'rolling_quantile_97_56', 'rolling_quantile_87.5_56', 'rolling_quantile_50_56', 'rolling_quantile_22.5_56', 'rolling_quantile_3_56', 'rolling_quantile_97_168', 'rolling_quantile_87.5_168', 'rolling_quantile_50_168', 'rolling_quantile_22.5_168', 'rolling_quantile_3_168']
    else:
        remove_features = ['id','state_id','store_id','date','wm_yr_wk','d',TARGET,'cluster',
                          'rolling_quantile_97_28', 'rolling_quantile_87.5_28', 'rolling_quantile_50_28', 'rolling_quantile_22.5_28', 'rolling_quantile_3_28', 'rolling_quantile_97_56', 'rolling_quantile_87.5_56', 'rolling_quantile_50_56', 'rolling_quantile_22.5_56', 'rolling_quantile_3_56', 'rolling_quantile_97_168', 'rolling_quantile_87.5_168', 'rolling_quantile_50_168', 'rolling_quantile_22.5_168', 'rolling_quantile_3_168']
    
    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # Skipping first n rows
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle('test_'+store_id+str(VER)+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]

In [10]:
def make_lag_roll(LAG_DAY,lag_df_new):
   
    lag_df = base_test[['id','d',TARGET]]
    
    lag_df=lag_df.sort_values(by=["d"])
  
    for i in range(0,len(LAG_DAY)):

        shift_day = LAG_DAY[i][0]
        roll_wind = LAG_DAY[i][1]
        col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
        lag_df[col_name] = (lag_df.groupby(['id'])[TARGET]).transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    lag_df_new=lag_df.drop(columns=["sales"])
    return lag_df_new

In [11]:
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    "lambda":0.1,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': -1,
                } 



# lgb_params ={
#         "objective" : "tweedie",
#         "metric" :"rmse",
#         "force_row_wise" : True,
#         "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
#         "sub_row" : 0.75,
#         "bagging_freq" : 1,
#         "lambda_l2" : 0.1,
#         "metric": ["rmse"],
#         "nthread": -1,
#         "tweedie_variance_power":1.1,
#     'verbosity': 1,
# #     'num_iterations' : 1500,
#     'num_leaves': 128,
#     "min_data_in_leaf": 104,
#     }




# Let's look closer on params

## 'boosting_type': 'gbdt'
# we have 'goss' option for faster training
# but it normally leads to underfit.
# Also there is good 'dart' mode
# but it takes forever to train
# and model performance depends 
# a lot on random factor 
# https://www.kaggle.com/c/home-credit-default-risk/discussion/60921

## 'objective': 'tweedie'
# Tweedie Gradient Boosting for Extremely
# Unbalanced Zero-inflated Data
# https://arxiv.org/pdf/1811.10192.pdf
# and many more articles about tweediie
#
# Strange (for me) but Tweedie is close in results
# to my own ugly loss.
# My advice here - make OWN LOSS function
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/140564
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/143070
# I think many of you already using it (after poisson kernel appeared) 
# (kagglers are very good with "params" testing and tuning).
# Try to figure out why Tweedie works.
# probably it will show you new features options
# or data transformation (Target transformation?).

## 'tweedie_variance_power': 1.1
# default = 1.5
# set this closer to 2 to shift towards a Gamma distribution
# set this closer to 1 to shift towards a Poisson distribution
# my CV shows 1.1 is optimal 
# but you can make your own choice

## 'metric': 'rmse'
# Doesn't mean anything to us
# as competition metric is different
# and we don't use early stoppings here.
# So rmse serves just for general 
# model performance overview.
# Also we use "fake" validation set
# (as it makes part of the training set)
# so even general rmse score doesn't mean anything))
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/133834

## 'subsample': 0.5
# Serves to fight with overfit
# this will randomly select part of data without resampling
# Chosen by CV (my CV can be wrong!)
# Next kernel will be about CV

##'subsample_freq': 1
# frequency for bagging
# default value - seems ok

## 'learning_rate': 0.03
# Chosen by CV
# Smaller - longer training
# but there is an option to stop 
# in "local minimum"
# Bigger - faster training
# but there is a chance to
# not find "global minimum" minimum

## 'num_leaves': 2**11-1
## 'min_data_in_leaf': 2**12-1
# Force model to use more features
# We need it to reduce "recursive"
# error impact.
# Also it leads to overfit
# that's why we use small 

# 'max_bin': 100
## l1, l2 regularizations
# https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c
# Good tiny explanation
# l2 can work with bigger num_leaves
# but my CV doesn't show boost
                    
## 'n_estimators': 1400
# CV shows that there should be
# different values for each state/store.
# Current value was chosen 
# for general purpose.
# As we don't use any early stopings
# careful to not overfit Public LB.

##'feature_fraction': 0.5
# LightGBM will randomly select 
# part of features on each iteration (tree).
# We have maaaany features
# and many of them are "duplicates"
# and many just "noise"
# good values here - 0.5-0.7 (by CV)

## 'boost_from_average': False
# There is some "problem"
# to code boost_from_average for 
# custom loss
# 'True' makes training faster
# BUT carefull use it
# https://github.com/microsoft/LightGBM/issues/1514

In [12]:
VER = 5                        # Our model version
SEED = 42                      # We want all things
seed_everything(SEED)            # to be as deterministic 
lgb_params['seed'] = SEED        # as possible
N_CORES = psutil.cpu_count()     # Available CPU cores


#LIMITS and const
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1941               # End day of our train set, change this part for final
P_HORIZON   = 28                 # Prediction horizon

#FEATURES to remove
## These features lead to overfit
## or values not present in test set
mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

#PATHS for Features
BASE     = 'grid_part_1.pkl'
PRICE    = 'grid_part_2.pkl'
CALENDAR = 'grid_part_3.pkl'
LAGS     = 'lags_df_28_v3.pkl'
MEAN_ENC = 'mean_encoding_df.pkl'


# AUX(pretrained) Models paths

#STORES ids
STORES_IDS = pd.read_csv('sales_train_evaluation.csv')['store_id']#change this part for final
STORES_IDS = list(STORES_IDS.unique())

#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,28,56]:
        ROLS_SPLIT.append([i,j])

In [13]:
for store_id in STORES_IDS:
    print('Train', store_id)
    
    # Get grid for current store
    grid_df, features_columns = get_data_by_store(store_id)
    
    print(features_columns)
    # Masks for 
    # Train (All data less than 1913)
    # "Validation" (Last 28 days - not real validation set)
    # Test (All data greater than 1913 day, 
    #       with some gap for recursive features)
    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
    preds_mask = grid_df['d']>(END_TRAIN-100)
    
    # Apply masks and save lgb dataset as bin
    # to reduce memory spikes during dtype convertations
    # https://github.com/Microsoft/LightGBM/issues/1032
    # "To avoid any conversions, you should always use np.float32"
    # or save to bin before start training
    # https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
    train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                            label=grid_df[train_mask][TARGET],
                            weight=grid_df[train_mask]['sell_price'])
    
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                            label=grid_df[valid_mask][TARGET],
                            weight=grid_df[valid_mask]['sell_price'])
    
    # Saving part of the dataset for later predictions
    # Removing features that we need to calculate recursively 
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle('test_'+store_id+str(VER)+'.pkl')
    del grid_df
    gc.collect()
    
    # Launch seeder again to make lgb training 100% deterministic
    # with each "code line" np.random "evolves" 
    # so we need (may want) to "reset" it
    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          valid_sets = [valid_data],
                          verbose_eval = 100,
                          )
    imp_type = "gain"
    features = estimator.feature_name()
    importances = estimator.feature_importance(imp_type)
    importance_df=pd.DataFrame(features,columns=['features'])
    importance_df['importances']=importances
    importance_df=importance_df.sort_values(by='importances', ascending=False)
    importance_df.to_csv(store_id+'_fe_imp_'+str(VER)+'.csv',index=False)
    del importance_df
    gc.collect()
    
    # Save model - it's not real '.bin' but a pickle file
    # estimator = lgb.Booster(model_file='model.txt')
    # can only predict with the best iteration (or the saving iteration)
    # pickle.dump gives us more flexibility
    # like estimator.predict(TEST, num_iteration=100)
    # num_iteration - number of iteration want to predict with, 
    # NULL or <= 0 means use best iteration
    model_name = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

    # Remove temporary files and objects 
    # to free some hdd space and ram memory
    #     !rm train_data.bin
    del train_data, valid_data, estimator
    gc.collect()

Train CA_1
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'temperature_high', 'temperature_con', 'rainfall_m', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend', 'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d', 'tm_w', 'tm_m', 'tm_q', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7',

[100]	valid_0's rmse: 1.19588
[200]	valid_0's rmse: 1.17163
[300]	valid_0's rmse: 1.16152
[400]	valid_0's rmse: 1.15556
[500]	valid_0's rmse: 1.15039
[600]	valid_0's rmse: 1.14642
[700]	valid_0's rmse: 1.14205
[800]	valid_0's rmse: 1.13865
[900]	valid_0's rmse: 1.13504
[1000]	valid_0's rmse: 1.13189
[1100]	valid_0's rmse: 1.1286
[1200]	valid_0's rmse: 1.12545
[1300]	valid_0's rmse: 1.12262
[1400]	valid_0's rmse: 1.11968
Train TX_2
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'temperature_high', 'temperature_con', 'rainfall_m', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend', 'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d', 'tm_w', 'tm_m', 'tm_q', 'tm

[100]	valid_0's rmse: 1.2945
[200]	valid_0's rmse: 1.26245
[300]	valid_0's rmse: 1.25459
[400]	valid_0's rmse: 1.24724
[500]	valid_0's rmse: 1.24181
[600]	valid_0's rmse: 1.23734
[700]	valid_0's rmse: 1.2327
[800]	valid_0's rmse: 1.22894
[900]	valid_0's rmse: 1.22522
[1000]	valid_0's rmse: 1.22143
[1100]	valid_0's rmse: 1.21836
[1200]	valid_0's rmse: 1.21494
[1300]	valid_0's rmse: 1.21171
[1400]	valid_0's rmse: 1.20897


In [14]:
# Create Dummy DataFrame to store predictions
all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

# Timer to measure predictions time 
main_time = time.time()

# Loop over each prediction day
# As rolling lags are the most timeconsuming
# we will calculate it for whole day


for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # Make temporary grid to calculate rolling lags
    grid_df = base_test.copy()
    
    
    lag_df_new = pd.DataFrame()

    lag_df_new=make_lag_roll(ROLS_SPLIT,lag_df_new)


    grid_df = grid_df.merge(lag_df_new, on=['id','d'], how='left')


    for store_id in STORES_IDS:
        
        if store_id in ['CA_1', 'CA_2', 'CA_3','CA_4','TX_1','TX_2','TX_3']:
            MODEL_FEATURES = ['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 
                              'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept',
                              'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 
                              'price_momentum_y', 'temperature_high', 'temperature_con', 'rainfall_m', 
                              'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 
                              'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend',
                              'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d',
                              'tm_w', 'tm_m', 'tm_q', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', 
                              'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 
                              'enc_item_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 
                              'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36',
                              'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 
                              'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 
                              'rolling_mean_28', 'rolling_std_28', 'rolling_mean_56', 'rolling_std_56', 
                              'rolling_mean_168', 'rolling_std_168', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14',
                              'rolling_mean_tmp_1_28', 'rolling_mean_tmp_1_56', 'rolling_mean_tmp_7_7', 
                              'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_28', 'rolling_mean_tmp_7_56', 
                              'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_28', 'rolling_mean_tmp_14_56']
        else:
            MODEL_FEATURES = ['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 
                              'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept',
                              'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 
                              'price_momentum_y', 'temperature_high', 'temperature_con', 'rainfall_m', 'snow_m',
                              'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 
                              'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend',
                              'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d',
                              'tm_w', 'tm_m', 'tm_q', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', 
                              'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 
                              'enc_item_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 
                              'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36',
                              'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 
                              'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 
                              'rolling_mean_28', 'rolling_std_28', 'rolling_mean_56', 'rolling_std_56', 
                              'rolling_mean_168', 'rolling_std_168', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14',
                              'rolling_mean_tmp_1_28', 'rolling_mean_tmp_1_56', 'rolling_mean_tmp_7_7', 
                              'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_28', 'rolling_mean_tmp_7_56', 
                              'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_28', 'rolling_mean_tmp_14_56']
        # Read all our models and make predictions
        # for each day/store pairs
        model_path = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin' 

        estimator = pickle.load(open(model_path, 'rb'))

        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        store_mask = base_test['store_id']==store_id

        mask = (day_mask)&(store_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])

    # Make good column naming and add 
    # to all_preds DataFrame
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()

    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    
    del temp_df, lag_df_new

all_preds = all_preds.reset_index(drop=True)
all_preds.head()

Predict | Day: 1
##########  5.03 min round |  5.03 min total |  40055.52 day sales |
Predict | Day: 2
##########  4.99 min round |  10.03 min total |  37631.61 day sales |
Predict | Day: 3
##########  5.01 min round |  15.03 min total |  37061.95 day sales |
Predict | Day: 4
##########  4.99 min round |  20.02 min total |  37333.67 day sales |
Predict | Day: 5
##########  4.99 min round |  25.01 min total |  42794.83 day sales |
Predict | Day: 6
##########  4.97 min round |  29.98 min total |  50641.78 day sales |
Predict | Day: 7
##########  5.00 min round |  34.98 min total |  51193.70 day sales |
Predict | Day: 8
##########  4.95 min round |  39.94 min total |  45988.98 day sales |
Predict | Day: 9
##########  5.00 min round |  44.94 min total |  38875.05 day sales |
Predict | Day: 10
##########  4.96 min round |  49.91 min total |  43856.73 day sales |
Predict | Day: 11
##########  4.97 min round |  54.88 min total |  45393.91 day sales |
Predict | Day: 12
##########  4.97 min rou

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.843942,0.757169,0.715683,0.753203,1.063534,1.382639,1.186249,1.013104,0.841206,...,0.920304,1.275342,1.245108,0.956381,0.854709,0.815611,0.909075,1.150540,1.216573,1.083584
1,HOBBIES_1_002_CA_1_evaluation,0.221871,0.180619,0.178077,0.204808,0.223516,0.262944,0.293602,0.244855,0.195481,...,0.266105,0.331042,0.354719,0.214150,0.239515,0.259978,0.260695,0.307596,0.381829,0.393854
2,HOBBIES_1_003_CA_1_evaluation,0.596188,0.517904,0.572299,0.586083,0.765112,0.849789,0.910160,0.606593,0.504223,...,0.688311,0.807232,0.793996,0.527569,0.471919,0.506101,0.538593,0.759713,0.788665,0.789337
3,HOBBIES_1_004_CA_1_evaluation,1.444556,1.349752,1.253506,1.340283,1.902976,2.460744,3.080803,1.857540,1.348157,...,1.792015,2.636771,3.110266,1.633284,1.367279,1.332755,1.293782,1.824124,2.430418,2.509399
4,HOBBIES_1_005_CA_1_evaluation,1.075127,0.949467,0.917487,0.949843,1.123287,1.412327,1.518267,1.282982,0.885294,...,1.117109,1.424336,1.485127,1.051679,0.967747,0.959812,1.014363,1.387304,1.579219,1.380943


In [15]:
all_preds.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30485,FOODS_3_823_WI_3_evaluation,0.502994,0.498345,0.419373,0.444788,0.555020,0.567369,0.652649,0.493585,0.512123,...,0.600543,0.751355,0.855521,0.582112,0.612754,0.567523,0.478615,0.523902,0.592237,0.614658
30486,FOODS_3_824_WI_3_evaluation,0.290350,0.296965,0.265807,0.258709,0.261291,0.323549,0.391024,0.277187,0.315013,...,0.308019,0.472592,0.496950,0.355754,0.383251,0.393335,0.338792,0.319868,0.368173,0.439551
30487,FOODS_3_825_WI_3_evaluation,0.670223,0.530063,0.502035,0.432171,0.507270,0.600266,0.659112,0.709735,0.510975,...,0.827585,1.240528,1.369278,0.950195,1.034824,0.988889,0.666889,0.725947,0.756110,0.866015
30488,FOODS_3_826_WI_3_evaluation,0.959349,1.089464,0.920924,0.973131,1.057325,1.205529,1.126409,1.245759,1.070541,...,1.031615,1.388608,1.433828,1.154460,1.285401,1.327883,1.148370,1.147581,1.205018,1.154119
30489,FOODS_3_827_WI_3_evaluation,1.887284,1.724988,1.714910,1.689012,2.094473,2.113791,1.911832,1.719376,1.457794,...,1.600495,1.885145,1.903074,1.608934,1.668094,1.789802,1.564792,1.694608,1.755475,1.737768


In [16]:
all_preds.shape

(30490, 29)

In [17]:
all_preds.describe()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
count,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,...,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000
mean,1.313726,1.234228,1.215544,1.224456,1.403569,1.660931,1.679032,1.508330,1.275010,1.438397,...,1.525667,1.914256,1.941946,1.500168,1.402132,1.478276,1.370720,1.540403,1.780806,1.640482
std,2.691813,2.525724,2.470556,2.507346,2.987191,3.483030,3.453098,3.122830,2.544430,2.844386,...,3.250733,4.053365,4.081101,3.051956,2.963852,3.121355,2.877687,3.346105,3.901129,3.606472
min,0.001767,0.012228,0.013233,0.015650,0.016275,0.015168,0.016093,0.013059,0.010825,0.012396,...,0.014171,0.016336,0.015275,0.009456,0.010623,0.012896,0.013712,0.015541,0.018222,0.016799
25%,0.263916,0.270608,0.267400,0.268604,0.300918,0.349483,0.356584,0.315771,0.279281,0.313666,...,0.316908,0.389265,0.392327,0.305049,0.286251,0.301061,0.290663,0.323446,0.364593,0.332571
50%,0.607223,0.581004,0.570779,0.573815,0.643954,0.765127,0.783891,0.698812,0.602675,0.675767,...,0.689501,0.861293,0.886676,0.688536,0.630425,0.661947,0.623222,0.692913,0.790574,0.745275
75%,1.324028,1.233730,1.222211,1.219653,1.384028,1.647071,1.690371,1.476255,1.275026,1.428240,...,1.483235,1.860074,1.913337,1.487870,1.368282,1.441254,1.333435,1.491846,1.721214,1.604548
max,106.723696,100.430102,98.623316,97.570936,138.743669,157.594060,147.585001,119.802856,99.659351,102.095870,...,141.072089,166.067940,173.168442,118.365546,107.313552,108.296565,111.203063,144.642447,171.092511,153.898552


In [18]:
# all the following is changed

In [19]:
# replace validation part
train_df = pd.read_csv('sales_train_evaluation.csv')
train_df=train_df[['id','d_1914','d_1915','d_1916','d_1917','d_1918','d_1919','d_1920','d_1921','d_1922','d_1923',
                  'd_1924','d_1925','d_1926','d_1927','d_1928','d_1929','d_1930','d_1931','d_1932','d_1933',
                  'd_1934','d_1935','d_1936','d_1937','d_1938','d_1939','d_1940','d_1941']]

In [20]:
train_df.head()

,id,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,0,0,0,2,0,3,5,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,0,1,0,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,0,0,1,1,0,2,1,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,0,0,1,2,4,1,6,4,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,1,0,2,3,1,0,3,2,3,...,0,0,0,2,1,0,0,2,1,0


In [21]:
submission = pd.read_csv('sample_submission.csv')

In [22]:
submission.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
submission.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60979,FOODS_3_827_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
train_df['id']=train_df['id'].str.replace('evaluation','validation')

In [25]:
train_df.head()

,id,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_validation,0,0,0,2,0,3,5,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_validation,0,1,0,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,1,1,0,2,1,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_validation,0,0,1,2,4,1,6,4,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_validation,1,0,2,3,1,0,3,2,3,...,0,0,0,2,1,0,0,2,1,0


In [26]:
train_df.columns=submission.columns

In [27]:
train_df.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,2,0,3,5,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_validation,0,1,0,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,1,1,0,2,1,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_validation,0,0,1,2,4,1,6,4,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_validation,1,0,2,3,1,0,3,2,3,...,0,0,0,2,1,0,0,2,1,0


In [28]:
train_df.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30485,FOODS_3_823_WI_3_validation,0,0,0,2,2,0,0,0,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_validation,0,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_validation,0,0,1,1,0,2,1,1,0,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_validation,1,3,0,1,2,1,0,2,1,...,1,1,1,4,6,0,1,1,1,0
30489,FOODS_3_827_WI_3_validation,0,0,0,0,0,1,1,1,2,...,1,2,0,5,4,0,2,2,5,1


In [29]:
train_df.shape

(30490, 29)

In [30]:
submission.shape

(60980, 29)

In [31]:
submission = submission[['id']]
sub1 = submission.merge(train_df, on=['id'], how='left')

In [32]:
sub1.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,2.0,0.0,3.0,5.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_validation,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0
3,HOBBIES_1_004_CA_1_validation,0.0,0.0,1.0,2.0,4.0,1.0,6.0,4.0,0.0,...,1.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,6.0
4,HOBBIES_1_005_CA_1_validation,1.0,0.0,2.0,3.0,1.0,0.0,3.0,2.0,3.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0


In [33]:
sub1.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60976,FOODS_3_824_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60977,FOODS_3_825_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60978,FOODS_3_826_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60979,FOODS_3_827_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
sub1=sub1[:30490]

In [35]:
sub1.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,2.0,0.0,3.0,5.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_validation,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0
3,HOBBIES_1_004_CA_1_validation,0.0,0.0,1.0,2.0,4.0,1.0,6.0,4.0,0.0,...,1.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,6.0
4,HOBBIES_1_005_CA_1_validation,1.0,0.0,2.0,3.0,1.0,0.0,3.0,2.0,3.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0


In [36]:
sub1.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30485,FOODS_3_823_WI_3_validation,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,2.0,...,1.0,0.0,3.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
30486,FOODS_3_824_WI_3_validation,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
30487,FOODS_3_825_WI_3_validation,0.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,0.0,...,0.0,0.0,1.0,2.0,0.0,1.0,0.0,1.0,0.0,2.0
30488,FOODS_3_826_WI_3_validation,1.0,3.0,0.0,1.0,2.0,1.0,0.0,2.0,1.0,...,1.0,1.0,1.0,4.0,6.0,0.0,1.0,1.0,1.0,0.0
30489,FOODS_3_827_WI_3_validation,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,...,1.0,2.0,0.0,5.0,4.0,0.0,2.0,2.0,5.0,1.0


In [37]:
sub2 = submission.merge(all_preds, on=['id'], how='left')

In [38]:
sub2.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
sub2.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0.502994,0.498345,0.419373,0.444788,0.555020,0.567369,0.652649,0.493585,0.512123,...,0.600543,0.751355,0.855521,0.582112,0.612754,0.567523,0.478615,0.523902,0.592237,0.614658
60976,FOODS_3_824_WI_3_evaluation,0.290350,0.296965,0.265807,0.258709,0.261291,0.323549,0.391024,0.277187,0.315013,...,0.308019,0.472592,0.496950,0.355754,0.383251,0.393335,0.338792,0.319868,0.368173,0.439551
60977,FOODS_3_825_WI_3_evaluation,0.670223,0.530063,0.502035,0.432171,0.507270,0.600266,0.659112,0.709735,0.510975,...,0.827585,1.240528,1.369278,0.950195,1.034824,0.988889,0.666889,0.725947,0.756110,0.866015
60978,FOODS_3_826_WI_3_evaluation,0.959349,1.089464,0.920924,0.973131,1.057325,1.205529,1.126409,1.245759,1.070541,...,1.031615,1.388608,1.433828,1.154460,1.285401,1.327883,1.148370,1.147581,1.205018,1.154119
60979,FOODS_3_827_WI_3_evaluation,1.887284,1.724988,1.714910,1.689012,2.094473,2.113791,1.911832,1.719376,1.457794,...,1.600495,1.885145,1.903074,1.608934,1.668094,1.789802,1.564792,1.694608,1.755475,1.737768


In [40]:
sub2=sub2[30490:]

In [41]:
sub2.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30490,HOBBIES_1_001_CA_1_evaluation,0.843942,0.757169,0.715683,0.753203,1.063534,1.382639,1.186249,1.013104,0.841206,...,0.920304,1.275342,1.245108,0.956381,0.854709,0.815611,0.909075,1.150540,1.216573,1.083584
30491,HOBBIES_1_002_CA_1_evaluation,0.221871,0.180619,0.178077,0.204808,0.223516,0.262944,0.293602,0.244855,0.195481,...,0.266105,0.331042,0.354719,0.214150,0.239515,0.259978,0.260695,0.307596,0.381829,0.393854
30492,HOBBIES_1_003_CA_1_evaluation,0.596188,0.517904,0.572299,0.586083,0.765112,0.849789,0.910160,0.606593,0.504223,...,0.688311,0.807232,0.793996,0.527569,0.471919,0.506101,0.538593,0.759713,0.788665,0.789337
30493,HOBBIES_1_004_CA_1_evaluation,1.444556,1.349752,1.253506,1.340283,1.902976,2.460744,3.080803,1.857540,1.348157,...,1.792015,2.636771,3.110266,1.633284,1.367279,1.332755,1.293782,1.824124,2.430418,2.509399
30494,HOBBIES_1_005_CA_1_evaluation,1.075127,0.949467,0.917487,0.949843,1.123287,1.412327,1.518267,1.282982,0.885294,...,1.117109,1.424336,1.485127,1.051679,0.967747,0.959812,1.014363,1.387304,1.579219,1.380943


In [42]:
sub2.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0.502994,0.498345,0.419373,0.444788,0.555020,0.567369,0.652649,0.493585,0.512123,...,0.600543,0.751355,0.855521,0.582112,0.612754,0.567523,0.478615,0.523902,0.592237,0.614658
60976,FOODS_3_824_WI_3_evaluation,0.290350,0.296965,0.265807,0.258709,0.261291,0.323549,0.391024,0.277187,0.315013,...,0.308019,0.472592,0.496950,0.355754,0.383251,0.393335,0.338792,0.319868,0.368173,0.439551
60977,FOODS_3_825_WI_3_evaluation,0.670223,0.530063,0.502035,0.432171,0.507270,0.600266,0.659112,0.709735,0.510975,...,0.827585,1.240528,1.369278,0.950195,1.034824,0.988889,0.666889,0.725947,0.756110,0.866015
60978,FOODS_3_826_WI_3_evaluation,0.959349,1.089464,0.920924,0.973131,1.057325,1.205529,1.126409,1.245759,1.070541,...,1.031615,1.388608,1.433828,1.154460,1.285401,1.327883,1.148370,1.147581,1.205018,1.154119
60979,FOODS_3_827_WI_3_evaluation,1.887284,1.724988,1.714910,1.689012,2.094473,2.113791,1.911832,1.719376,1.457794,...,1.600495,1.885145,1.903074,1.608934,1.668094,1.789802,1.564792,1.694608,1.755475,1.737768


In [43]:
final_sub=pd.concat([sub1,sub2],axis=0)

In [44]:
final_sub.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,2.0,0.0,3.0,5.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_validation,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0
3,HOBBIES_1_004_CA_1_validation,0.0,0.0,1.0,2.0,4.0,1.0,6.0,4.0,0.0,...,1.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,6.0
4,HOBBIES_1_005_CA_1_validation,1.0,0.0,2.0,3.0,1.0,0.0,3.0,2.0,3.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0


In [45]:
final_sub.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0.502994,0.498345,0.419373,0.444788,0.555020,0.567369,0.652649,0.493585,0.512123,...,0.600543,0.751355,0.855521,0.582112,0.612754,0.567523,0.478615,0.523902,0.592237,0.614658
60976,FOODS_3_824_WI_3_evaluation,0.290350,0.296965,0.265807,0.258709,0.261291,0.323549,0.391024,0.277187,0.315013,...,0.308019,0.472592,0.496950,0.355754,0.383251,0.393335,0.338792,0.319868,0.368173,0.439551
60977,FOODS_3_825_WI_3_evaluation,0.670223,0.530063,0.502035,0.432171,0.507270,0.600266,0.659112,0.709735,0.510975,...,0.827585,1.240528,1.369278,0.950195,1.034824,0.988889,0.666889,0.725947,0.756110,0.866015
60978,FOODS_3_826_WI_3_evaluation,0.959349,1.089464,0.920924,0.973131,1.057325,1.205529,1.126409,1.245759,1.070541,...,1.031615,1.388608,1.433828,1.154460,1.285401,1.327883,1.148370,1.147581,1.205018,1.154119
60979,FOODS_3_827_WI_3_evaluation,1.887284,1.724988,1.714910,1.689012,2.094473,2.113791,1.911832,1.719376,1.457794,...,1.600495,1.885145,1.903074,1.608934,1.668094,1.789802,1.564792,1.694608,1.755475,1.737768


In [46]:
final_sub.describe()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
count,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,...,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000
mean,1.293023,1.199059,1.172630,1.181792,1.405228,1.657442,1.709178,1.462282,1.364202,1.368608,...,1.485579,1.847715,1.953607,1.444771,1.336963,1.347469,1.291510,1.468004,1.735237,1.711320
std,2.977283,2.750601,2.693766,2.703040,3.308784,3.792296,3.743211,3.289780,3.135574,3.087898,...,3.459946,4.244548,4.404778,3.185761,3.046768,3.064218,2.918112,3.431978,3.996503,3.960547
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.059955,0.000000,0.000000,0.000000,...,0.000000,0.049966,0.098634,0.000000,0.000000,0.000000,0.000000,0.000000,0.026488,0.070336
50%,0.478479,0.447163,0.435931,0.433477,0.558496,0.745594,0.803703,0.606618,0.529792,0.552365,...,0.603001,0.871782,0.954694,0.603935,0.529619,0.535474,0.507884,0.609221,0.791397,0.779519
75%,1.245770,1.099047,1.071931,1.065976,1.402298,1.885902,2.000000,1.505995,1.288839,1.345675,...,1.505782,2.000000,2.000000,1.506099,1.300996,1.321619,1.221899,1.483076,2.000000,2.000000
max,133.000000,117.000000,113.000000,99.000000,196.000000,157.594060,147.585001,119.802856,143.000000,107.000000,...,143.000000,166.067940,187.000000,118.365546,107.313552,108.296565,111.203063,144.642447,171.092511,153.898552


In [47]:
final_sub.to_csv('lgb_bystore_final3.csv',index=False)